# 本日の概要
- 本日話すこと
    - J-Quants APIの利用の仕方
    - J-Quants APIを利用した簡単なデータ分析

- 本日話さないこと
    - ゴリゴリのデータ分析
    - データ分析を利用した投資・投資手法の紹介等


プログラミング初心者の方や、これからデータ分析を始める方向けに、  
コードも交えてJ-Quants APIの利用方法について紹介させていただきます！！

# J-QuantsAPIについて

## J-Quants APIの利用の仕方



1. [APIサービス登録ページ](https://application.jpx-jquants.com/register)よりJ-Quants APIサービスへ登録

2. 登録後、[ログインページ](https://application.jpx-jquants.com/)から登録したメールアドレス及びパスワードでJ-Quants APIサービスへログインしリフレッシュトークンを取得（取得ボタンを押下すると、自動的にクリップボードへコピーされます）
3. CurlコマンドやPython等を用いて、リフレッシュトークンを利用し、ID tokenを取得
4. CurlコマンドやPython等を用いて、取得したID tokenを利用し、データを取得

なお、J-Quants APIの各種APIの仕様は[こちら](https://jpx.gitbook.io/j-quants-api/api-reference)をご参照ください。

### 上記3. について、pythonによるID token取得サンプル

In [1]:
import requests
import json

REFRESH_TOKEN = "こちらにクリップボードにコピーしたリフレッシュトークンを貼り付け"
idToken = requests.post(
    f"https://api.jpx-jquants.com/v1/token/auth_refresh?refreshtoken={REFRESH_TOKEN}"
).json()["idToken"]


### 上記4.について、 pythonによるデータ取得サンプル（銘柄情報）

In [ ]:
# 上記３.idTokenを取得していることが前提です
headers = {'Authorization': 'Bearer {}'.format(idToken)}
r = requests.get("https://api.jpx-jquants.com/v1/listed/info", headers=headers)

# APIからのresponseはjson形式
info_data = r.json()["info"]

# 取得したデータを5つ表示
info_data[:5]


## J-Quants API各種クライアントライブラリ

J-Quants APIにはより使いやすくなるように、クライアントライブラリをalpacaさんや有志の方に作製いただいております！  
作製いただいた方ありがとうございます！！



* Python用クライアントライブラリ（ [jquants-api-client-python](https://github.com/J-Quants/jquants-api-client-python) ）

* R用クライアントライブラリ（ [JQuantsR](https://github.com/J-Quants/JQuantsR) ）

* Julia用クライアントライブラリ（ [JQuants.jl](https://github.com/ki-chi/JQuants.jl) ）

今回はPythonを用いるため、Python用クライアントライブラリを利用します。

In [2]:
# Google Colabでjquants-api-client-pythonを使えるようにインストール
!pip install jquants-api-client


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Driveをマウント後、[こちら](https://github.com/J-Quants/jquants-api-client-python#%E8%A8%AD%E5%AE%9A)を参考にAPI利用のための設定ファイル（jquants-api.toml）を作成します。

この設定を行うことで、jquantsapiパッケージを利用し、リフレッシュトークン及びtokenIDを自動で取得してくれます！！

In [4]:
import requests
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import jquantsapi

# pandasデータフレームの描画設定
pd.set_option("max_columns", None)
pd.set_option("max_rows", 40)
pd.set_option("max_colwidth", 80)

# 今回はID及びPWは上記に記載の方法により別ファイルに保存しているため、引数特になしでOK。
cli = jquantsapi.Client()

# 以下のように直接引数として渡してもOKですが、メールアドレスとパスワードが平文でnotebookに残るため、別で設定ファイルを作成したほうが安全です。
# cli = jquantsapi.Client(mail_address="ご登録いただいたメールアドレス", password="ご登録いただいたパスワード")


In [6]:
# 試しに銘柄情報を取得
cli.get_listed_info().reset_index(drop=True)


,Code,CompanyName,CompanyNameEnglish,CompanyNameFull,SectorCode,UpdateDate,MarketCode
0,13010,極洋,"KYOKUYO CO.,LTD.",（株）極洋,0050,2022-09-28,A
1,13050,ＥＴＦ・ＴＰＸ,Daiwa ETF-TOPIX,大和アセットマネジメント株式会社 ダイワ上場投信−トピックス,9999,2022-09-28,5
2,13060,ＮＦＴＯＰＩＸ,NEXT FUNDS TOPIX Exchange Traded Fund,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,9999,2022-09-28,5
3,13080,上場ＴＰＸ,Nikko Exchange Traded Index Fund TOPIX,日興アセットマネジメント株式会社 上場インデックスファンドＴＯＰＩＸ,9999,2022-09-28,5
4,13090,ＮＦ中国株上証５０,NEXT FUNDS ChinaAMC SSE50 Index Exchange Traded Fund,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,9999,2022-09-28,5
...,...,...,...,...,...,...,...
4233,99930,ヤマザワ,"YAMAZAWA CO.,LTD.",（株）ヤマザワ,6100,2022-09-28,B
4234,99940,やまや,YAMAYA CORPORATION,（株）やまや,6100,2022-09-28,B
4235,99950,グローセル,"GLOSEL Co.,Ltd.",（株）グローセル,6050,2022-09-28,A
4236,99960,サトー商会,"Satoh&Co.,Ltd.",（株）サトー商会,6050,2022-09-28,B


その他、jquants-api-clientで利用できるものは[こちらの説明](https://github.com/J-Quants/jquants-api-client-python#%E5%AF%BE%E5%BF%9Capi)をご参照ください。


# 銘柄一覧情報 API（毎営業日の21時頃データ更新）
（注）2022/9/27時点の情報です。

銘柄情報は、東証・名証・札証・福証へ上場している銘柄について、銘柄情報適用年月日時点の情報を提供しています。

**過去時点の銘柄一覧情報や、上場廃止された銘柄については、現在提供しておりません。**

なお、市場コードについて「9（上場廃止）」となっている銘柄は、東証とその他の地取に重複上場していたものの、その後東証で上場廃止となり、その他の地取において単独上場となっている銘柄となります。

## API出力データ概要

|変数名|概要|
|---|---|
|Code|株式銘柄コード（5桁表示）|
|CompanyName|企業名略称|
|CompanyNameEnglish|英語の企業名|
|CompanyNameFull|企業名全文|
|SectorCode|33業種コード|
|UpdateDate|銘柄情報適用年月日|
|MarketCode|市場区分コード|

In [7]:
df_info = cli.get_listed_info().reset_index(drop=True)

# データフレームを確認
df_info.info()

print("\n")

# データフレームを描画
df_info


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4238 entries, 0 to 4237
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Code                4238 non-null   object        
 1   CompanyName         4238 non-null   object        
 2   CompanyNameEnglish  4238 non-null   object        
 3   CompanyNameFull     4238 non-null   object        
 4   SectorCode          4238 non-null   object        
 5   UpdateDate          4238 non-null   datetime64[ns]
 6   MarketCode          4238 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 231.9+ KB




,Code,CompanyName,CompanyNameEnglish,CompanyNameFull,SectorCode,UpdateDate,MarketCode
0,13010,極洋,"KYOKUYO CO.,LTD.",（株）極洋,0050,2022-09-28,A
1,13050,ＥＴＦ・ＴＰＸ,Daiwa ETF-TOPIX,大和アセットマネジメント株式会社 ダイワ上場投信−トピックス,9999,2022-09-28,5
2,13060,ＮＦＴＯＰＩＸ,NEXT FUNDS TOPIX Exchange Traded Fund,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,9999,2022-09-28,5
3,13080,上場ＴＰＸ,Nikko Exchange Traded Index Fund TOPIX,日興アセットマネジメント株式会社 上場インデックスファンドＴＯＰＩＸ,9999,2022-09-28,5
4,13090,ＮＦ中国株上証５０,NEXT FUNDS ChinaAMC SSE50 Index Exchange Traded Fund,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,9999,2022-09-28,5
...,...,...,...,...,...,...,...
4233,99930,ヤマザワ,"YAMAZAWA CO.,LTD.",（株）ヤマザワ,6100,2022-09-28,B
4234,99940,やまや,YAMAYA CORPORATION,（株）やまや,6100,2022-09-28,B
4235,99950,グローセル,"GLOSEL Co.,Ltd.",（株）グローセル,6050,2022-09-28,A
4236,99960,サトー商会,"Satoh&Co.,Ltd.",（株）サトー商会,6050,2022-09-28,B


In [8]:
# 欠損値の確認
df_info[df_info.isnull().any(axis=1)]


,Code,CompanyName,CompanyNameEnglish,CompanyNameFull,SectorCode,UpdateDate,MarketCode


### ひとくちメモ集

- 株式銘柄コードは、(発行体ごとに付番される固有名コード4桁)及び(株券の種類ごとに付番される予備コード1桁)で構成されている。  
（ex.普通株は末尾が0、優先株は末尾が5）


In [9]:
# 普通株と優先株
df_info[df_info["Code"].isin(["25930", "25935"])]


,Code,CompanyName,CompanyNameEnglish,CompanyNameFull,SectorCode,UpdateDate,MarketCode
568,25930,伊藤園,"ITO EN,LTD.",（株）伊藤園,3050,2022-09-28,A
569,25935,優−伊藤園,"Prf-ITO EN,LTD.",優−（株）伊藤園,3050,2022-09-28,A


なお、現在株式銘柄コードは数値のみですが、2024年1月1日以降に新たに設定するコード（新規上場等）から数字のほかに英文字が追加される予定とのことです。  
ご興味ある方は[こちら](https://www.jpx.co.jp/sicc/securities-code/index.html)に資料等があります。

- SectorCode(33業種コード)は、[こちら](https://www.jpx.co.jp/sicc/sectors/nlsgeu00000329wk-att/gyousyu.pdf)をご参照ください。  
（なお、`jquants-api-client`の`get_list()`では、業種名及び市場区分名を結合したデータを提供してくれます！！感謝の極み！！）


In [10]:
cli.get_list()


,Code,CompanyName,CompanyNameEnglish,CompanyNameFull,SectorCode,UpdateDate,MarketCode,SectorName,MarketName
0,13010,極洋,"KYOKUYO CO.,LTD.",（株）極洋,0050,2022-09-28,A,水産・農林業,プライム
1,13050,ＥＴＦ・ＴＰＸ,Daiwa ETF-TOPIX,大和アセットマネジメント株式会社 ダイワ上場投信−トピックス,9999,2022-09-28,5,(その他),その他
2,13060,ＮＦＴＯＰＩＸ,NEXT FUNDS TOPIX Exchange Traded Fund,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,9999,2022-09-28,5,(その他),その他
3,13080,上場ＴＰＸ,Nikko Exchange Traded Index Fund TOPIX,日興アセットマネジメント株式会社 上場インデックスファンドＴＯＰＩＸ,9999,2022-09-28,5,(その他),その他
4,13090,ＮＦ中国株上証５０,NEXT FUNDS ChinaAMC SSE50 Index Exchange Traded Fund,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,9999,2022-09-28,5,(その他),その他
...,...,...,...,...,...,...,...,...,...
4233,99930,ヤマザワ,"YAMAZAWA CO.,LTD.",（株）ヤマザワ,6100,2022-09-28,B,小売業,スタンダード
4234,99940,やまや,YAMAYA CORPORATION,（株）やまや,6100,2022-09-28,B,小売業,スタンダード
4235,99950,グローセル,"GLOSEL Co.,Ltd.",（株）グローセル,6050,2022-09-28,A,卸売業,プライム
4236,99960,サトー商会,"Satoh&Co.,Ltd.",（株）サトー商会,6050,2022-09-28,B,卸売業,スタンダード


- SecterCodeが9999のものは、ETFやREIT、インフラファンド等の銘柄になっています。

In [11]:
df_info[df_info["SectorCode"] == "9999"]


,Code,CompanyName,CompanyNameEnglish,CompanyNameFull,SectorCode,UpdateDate,MarketCode
1,13050,ＥＴＦ・ＴＰＸ,Daiwa ETF-TOPIX,大和アセットマネジメント株式会社 ダイワ上場投信−トピックス,9999,2022-09-28,5
2,13060,ＮＦＴＯＰＩＸ,NEXT FUNDS TOPIX Exchange Traded Fund,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ連動型上場投信,9999,2022-09-28,5
3,13080,上場ＴＰＸ,Nikko Exchange Traded Index Fund TOPIX,日興アセットマネジメント株式会社 上場インデックスファンドＴＯＰＩＸ,9999,2022-09-28,5
4,13090,ＮＦ中国株上証５０,NEXT FUNDS ChinaAMC SSE50 Index Exchange Traded Fund,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＣｈｉｎａＡＭＣ・中国株式・上証５０連動型上場投信,9999,2022-09-28,5
5,13110,ＮＦＴＰＸ３０,NEXT FUNDS TOPIX Core 30 Exchange Traded Fund,野村アセットマネジメント株式会社 ＮＥＸＴ ＦＵＮＤＳ ＴＯＰＩＸ Ｃｏｒｅ ３０連動型上場投信,9999,2022-09-28,5
...,...,...,...,...,...,...,...
3922,92820,Ｉ−いちごグリーン,Ichigo Green Infrastructure Investment Corporation,いちごグリーンインフラ投資法人,9999,2022-09-28,5
3923,92840,Ｉ−カナディアンソラ,"Canadian Solar Infrastructure Fund,Inc.",カナディアン・ソーラー・インフラ投資法人,9999,2022-09-28,5
3924,92850,Ｉ−東京インフラＦ,Tokyo Infrastructure Energy Investment Corporation,東京インフラ・エネルギー投資法人,9999,2022-09-28,5
3925,92860,Ｉ−エネクスインフラ,Enex Infrastructure Investment Corporation,エネクス・インフラ投資法人,9999,2022-09-28,5


## 若干のデータ分析

In [ ]:
# 銘柄一覧情報を取得
df_info = cli.get_list()


### 33業種別の企業数

In [ ]:
# ETF, REIT, インフラファンド等の33業種以外のデータを除外
df = df_info[df_info["SectorCode"] != "9999"]

# 33業種名でグループ化
df_sector = df.groupby("SectorName").size().to_frame(
    "Count").reset_index().sort_values(by="Count", ascending=False)
px.bar(df_sector, x="SectorName", y="Count", title="33業種別の企業数")


### 市場区分別の企業数

In [ ]:
# 市場区分名でグループ化
df_market = df_info.groupby("MarketName").size().to_frame(
    "Count").reset_index().sort_values(by="Count", ascending=False)
px.bar(df_market, x="MarketName", y="Count", title="市場区分別の企業数")


### 上記分析の両方を加味

In [ ]:
# ETF, REIT, インフラファンド等の33業種以外のデータを除外
df = df_info[df_info["SectorCode"] != "9999"]

# さらにプライム、スタンダード、グロース及びTokyoProMarket銘柄にフィルタ
df = df[df["MarketName"].isin(["プライム", "スタンダード", "グロース", "TOKYO PRO Market"])]

# 業種別及び市場名でグルーピング
df_sector_market = df.groupby([
    "SectorName", "MarketName"
]).size().to_frame("Count").reset_index().sort_values(by="Count",
                                                      ascending=False)
px.bar(df_sector_market,
       x="SectorName",
       y="Count",
       color="MarketName",
       barmode="relative",
       title="33業種及び市場区分別の企業数")


# 株価情報 API（毎営業日の18時30分頃データ更新）

（注）2022/9/27時点の情報です。

2017年1月から直近までの株価データを取得することができます。  
株価及び取引高は、株式分割・併合を考慮した調整済み株価（小数点第２位四捨五入）と調整前の株価の両方を取得することができます。  

なお、データを取得する際には、株式銘柄コード（code）または日付（date）の指定が必須となっています。

## API出力データ概要

|変数名|概要|
|---|---|
|Code|株式銘柄コード（5桁表示）|
|Date|取引日|
|Open|始値（調整前）|
|High|高値（調整前）|
|Low|安値（調整前）|
|Close|終値（調整前）|
|Volume|取引高（調整前）|
|TurnoverValue|取引代金|
|AdjustmentFactor|調整係数|
|AdjustmentOpen|調整済み始値|
|AdjustmentHigh|調整済み高値|
|AdjustmentLow|調整済み安値|
|AdjustmentClose|調整済み終値|
|AdjustmentVolume|調整済み取引高|

### 株式銘柄コード指定Ver

In [12]:
df_quotes_bycode = cli.get_prices_daily_quotes(code=8697)

# データフレームの情報を取得
df_quotes_bycode.info()

print("\n")

# データフレームを描画
df_quotes_bycode


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1400 entries, 0 to 1399
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Code              1400 non-null   object        
 1   Date              1400 non-null   datetime64[ns]
 2   Open              1399 non-null   float64       
 3   High              1399 non-null   float64       
 4   Low               1399 non-null   float64       
 5   Close             1399 non-null   float64       
 6   Volume            1399 non-null   float64       
 7   TurnoverValue     1399 non-null   float64       
 8   AdjustmentFactor  1400 non-null   float64       
 9   AdjustmentOpen    1399 non-null   float64       
 10  AdjustmentHigh    1399 non-null   float64       
 11  AdjustmentLow     1399 non-null   float64       
 12  AdjustmentClose   1399 non-null   float64       
 13  AdjustmentVolume  1399 non-null   float64       
dtypes: datetime64[ns](1), fl

,Code,Date,Open,High,Low,Close,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
0,86970,2017-01-04,1690.0,1713.0,1681.0,1706.0,1999000.0,3.401429e+09,1.0,1690.0,1713.0,1681.0,1706.0,1999000.0
1,86970,2017-01-05,1703.0,1716.0,1696.0,1712.0,2137100.0,3.653718e+09,1.0,1703.0,1716.0,1696.0,1712.0,2137100.0
2,86970,2017-01-06,1694.0,1698.0,1688.0,1691.0,1678400.0,2.841347e+09,1.0,1694.0,1698.0,1688.0,1691.0,1678400.0
3,86970,2017-01-10,1682.0,1688.0,1662.0,1664.0,1883600.0,3.147536e+09,1.0,1682.0,1688.0,1662.0,1664.0,1883600.0
4,86970,2017-01-11,1700.0,1702.0,1682.0,1687.0,1786100.0,3.019238e+09,1.0,1700.0,1702.0,1682.0,1687.0,1786100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,86970,2022-09-20,2035.0,2048.5,2018.0,2046.0,1119900.0,2.282960e+09,1.0,2035.0,2048.5,2018.0,2046.0,1119900.0
1396,86970,2022-09-21,2027.0,2048.5,2013.0,2015.0,925900.0,1.872475e+09,1.0,2027.0,2048.5,2013.0,2015.0,925900.0
1397,86970,2022-09-22,1997.0,2032.0,1993.5,2027.0,1155000.0,2.327555e+09,1.0,1997.0,2032.0,1993.5,2027.0,1155000.0
1398,86970,2022-09-26,1989.0,1992.5,1953.0,1957.5,1918600.0,3.768522e+09,1.0,1989.0,1992.5,1953.0,1957.5,1918600.0


In [13]:
# 欠損値の確認
df_quotes_bycode[df_quotes_bycode.isnull().any(axis=1)]


,Code,Date,Open,High,Low,Close,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
913,86970,2020-10-01,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


2020/10/1は、システム障害により終日売買停止のため、NaNが入っている。

### 取引日指定Ver

In [14]:
df_quotes_bydate = cli.get_prices_daily_quotes(date_yyyymmdd=20220801)

# データフレームの情報を取得
df_quotes_bydate.info()

print("\n")

# データフレームを描画
df_quotes_bydate


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4196 entries, 0 to 4195
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Code              4196 non-null   object        
 1   Date              4196 non-null   datetime64[ns]
 2   Open              4067 non-null   float64       
 3   High              4067 non-null   float64       
 4   Low               4067 non-null   float64       
 5   Close             4067 non-null   float64       
 6   Volume            4067 non-null   float64       
 7   TurnoverValue     4067 non-null   float64       
 8   AdjustmentFactor  4196 non-null   float64       
 9   AdjustmentOpen    4067 non-null   float64       
 10  AdjustmentHigh    4067 non-null   float64       
 11  AdjustmentLow     4067 non-null   float64       
 12  AdjustmentClose   4067 non-null   float64       
 13  AdjustmentVolume  4067 non-null   float64       
dtypes: datetime64[ns](1), fl

,Code,Date,Open,High,Low,Close,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
0,13010,2022-08-01,3630.0,3650.0,3615.0,3650.0,10600.0,3.860450e+07,1.0,3630.0,3650.0,3615.0,3650.0,10600.0
1,13050,2022-08-01,2024.5,2043.0,2019.5,2041.0,69750.0,1.421875e+08,1.0,2024.5,2043.0,2019.5,2041.0,69750.0
2,13060,2022-08-01,2001.0,2020.5,1996.5,2019.0,1645480.0,3.311800e+09,1.0,2001.0,2020.5,1996.5,2019.0,1645480.0
3,13080,2022-08-01,1979.5,1997.5,1972.5,1996.0,240200.0,4.784298e+08,1.0,1979.5,1997.5,1972.5,1996.0,240200.0
4,13090,2022-08-01,42840.0,43100.0,42210.0,42780.0,87.0,3.716680e+06,1.0,42840.0,43100.0,42210.0,42780.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4191,99930,2022-08-01,1420.0,1422.0,1418.0,1420.0,10800.0,1.532970e+07,1.0,1420.0,1422.0,1418.0,1420.0,10800.0
4192,99940,2022-08-01,2624.0,2662.0,2624.0,2662.0,5000.0,1.322050e+07,1.0,2624.0,2662.0,2624.0,2662.0,5000.0
4193,99950,2022-08-01,425.0,425.0,410.0,410.0,129300.0,5.362010e+07,1.0,425.0,425.0,410.0,410.0,129300.0
4194,99960,2022-08-01,1240.0,1241.0,1235.0,1241.0,3300.0,4.091000e+06,1.0,1240.0,1241.0,1235.0,1241.0,3300.0


In [15]:
# 欠損値の確認
df_quotes_bydate[df_quotes_bydate.isnull().any(axis=1)]


,Code,Date,Open,High,Low,Close,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
8,13190,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
12,13240,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
46,13870,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
54,13970,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
68,14320,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4064,97200,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4095,97800,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4129,98570,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4173,99670,2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


その日取引が成立しなかった銘柄については、NaNが入っている。

### ひとくちメモ集

- 株価情報APIは、`code`を指定していれば、`from`, `to`でのデータ取得も可能です。

In [ ]:
# JPXの株式について2022年8月1日から9月1日までのデータを取得
cli.get_prices_daily_quotes(code=86970,
                            from_yyyymmdd=20220801,
                            to_yyyymmdd=20220901)


,Code,Date,Open,High,Low,Close,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
0,86970,2022-08-01,2115.5,2144.0,2097.0,2143.5,945400.0,2.016768e+09,1.0,2115.5,2144.0,2097.0,2143.5,945400.0
1,86970,2022-08-02,2132.0,2136.0,2099.0,2102.5,857200.0,1.807583e+09,1.0,2132.0,2136.0,2099.0,2102.5,857200.0
2,86970,2022-08-03,2125.0,2143.5,2121.5,2129.0,941200.0,2.004944e+09,1.0,2125.0,2143.5,2121.5,2129.0,941200.0
3,86970,2022-08-04,2142.0,2142.5,2125.0,2138.0,691500.0,1.477278e+09,1.0,2142.0,2142.5,2125.0,2138.0,691500.0
4,86970,2022-08-05,2150.0,2171.0,2146.0,2164.5,775400.0,1.676780e+09,1.0,2150.0,2171.0,2146.0,2164.5,775400.0
5,86970,2022-08-08,2166.0,2174.0,2155.5,2171.5,712700.0,1.544867e+09,1.0,2166.0,2174.0,2155.5,2171.5,712700.0
6,86970,2022-08-09,2175.5,2181.5,2147.5,2155.0,757300.0,1.636470e+09,1.0,2175.5,2181.5,2147.5,2155.0,757300.0
7,86970,2022-08-10,2161.5,2169.5,2136.5,2147.0,676800.0,1.453722e+09,1.0,2161.5,2169.5,2136.5,2147.0,676800.0
8,86970,2022-08-12,2208.5,2209.0,2162.5,2172.0,1649500.0,3.602564e+09,1.0,2208.5,2209.0,2162.5,2172.0,1649500.0
9,86970,2022-08-15,2173.0,2193.5,2164.0,2191.5,645700.0,1.410109e+09,1.0,2173.0,2193.5,2164.0,2191.5,645700.0


- なお、地取に単独上場している銘柄については、株価配信していません。

In [ ]:
# 銘柄一覧について地取単独上場銘柄でフィルタ
not_tse_codes = df_info[df_info["MarketCode"] == ""]["Code"]

# ある日付の株価データを地取単独上場銘柄一覧でフィルタ
df_quotes_bydate[df_quotes_bydate["Code"].isin(not_tse_codes)]


,Code,Date,Open,High,Low,Close,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume


- jquants-api-clientでは、`get_price_range()`を使うことで、銘柄指定せずに`from`, `to`で株価データが取得できます！（ありがたい...！）

In [16]:
df_quotes = cli.get_price_range(start_dt="20220801", end_dt="20220831")
df_quotes


,Code,Date,Open,High,Low,Close,Volume,TurnoverValue,AdjustmentFactor,AdjustmentOpen,AdjustmentHigh,AdjustmentLow,AdjustmentClose,AdjustmentVolume
0,13010,2022-08-01,3630.0,3650.0,3615.0,3650.0,10600.0,38604500.0,1.0,3630.0,3650.0,3615.0,3650.0,10600.0
0,13010,2022-08-02,3650.0,3650.0,3610.0,3615.0,9200.0,33328500.0,1.0,3650.0,3650.0,3610.0,3615.0,9200.0
0,13010,2022-08-03,3600.0,3600.0,3570.0,3575.0,10600.0,37995500.0,1.0,3600.0,3600.0,3570.0,3575.0,10600.0
0,13010,2022-08-04,3600.0,3600.0,3505.0,3525.0,14500.0,51253500.0,1.0,3600.0,3600.0,3505.0,3525.0,14500.0
0,13010,2022-08-05,3510.0,3750.0,3510.0,3675.0,66900.0,245070500.0,1.0,3510.0,3750.0,3510.0,3675.0,66900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4194,99970,2022-08-25,745.0,752.0,738.0,748.0,163600.0,122059300.0,1.0,745.0,752.0,738.0,748.0,163600.0
4194,99970,2022-08-26,749.0,756.0,749.0,752.0,161100.0,121237800.0,1.0,749.0,756.0,749.0,752.0,161100.0
4193,99970,2022-08-29,737.0,745.0,735.0,742.0,134700.0,99769600.0,1.0,737.0,745.0,735.0,742.0,134700.0
4191,99970,2022-08-30,747.0,751.0,743.0,750.0,134600.0,100634800.0,1.0,747.0,751.0,743.0,750.0,134600.0


## 若干のデータ分析

- 株価分析でよくある図を作成してみる

In [ ]:
# JPXの2017年からの株価を取得
df_jpx = cli.get_prices_daily_quotes(code=8697)

# グラフに第2軸を設定
fig = make_subplots(specs=[[{"secondary_y": True}]])

# ローソク足を描画
fig.add_trace(
    go.Candlestick(x=df_jpx.Date,
                   open=df_jpx.AdjustmentOpen,
                   high=df_jpx.AdjustmentHigh,
                   low=df_jpx.AdjustmentLow,
                   close=df_jpx.AdjustmentClose,
                   name="OHLC"))

# 25日移動平均
fig.add_trace(
    go.Scatter(x=df_jpx.Date,
               y=df_jpx.AdjustmentClose.rolling(25).mean(),
               name="25日移動平均線"))

# 75日移動平均
fig.add_trace(
    go.Scatter(x=df_jpx.Date,
               y=df_jpx.AdjustmentClose.rolling(75).mean(),
               name="75日移動平均線"))

# 取引高を第2軸に設定
fig.add_trace(go.Bar(x=df_jpx.Date, y=df_jpx.AdjustmentVolume, name="調整済み取引高"),
              secondary_y=True)
fig.update_layout(title_text="JPX（8697）のローソク足及び出来高")
fig.show()


# 財務情報 API（18時ごろ及び24時30分頃更新）
（注）2022/9/27時点の情報です。

財務情報は、2017年から直近までの上場会社の四半期及び通期の決算短信に係る情報を取得できます。

## API出力データ概要


|変数名|概要|
|---|---|
|DisclosureNumber|適時開示番号（TDnet内で一意になるコード）|
|DisclosedDate|適時開示された日|
|ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements|四半期財務諸表の作成に特有の会計処理の適用|
|AverageNumberOfShares|会計期中の平均株式数|
|BookValuePerShare|一株あたりの純資産|
|ChangesBasedOnRevisionsOfAccountingStandard|会計基準等の改正に伴う会計方針の変更|
|ChangesInAccountingEstimates|会計上の見積りの変更
|
|ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard|会計基準等の改正に伴う変更以外の会計方針の変更|
|CurrentFiscalYearEndDate|当事業年度終了日|
|CurrentFiscalYearStartDate|当事業年度開始日|
|CurrentPeriodEndDate|当会計期間終了日|
|DisclosedTime|開示時刻|
|DisclosedUnixTime|開示UnixTime|
|EarningsPerShare|1株当たり当期純利益|
|Equity|純資産|
|EquityToAssetRatio|自己資本比率|
|ForecastDividendPerShare1stQuarter|第1四半期末の配当予想|
|ForecastDividendPerShare2ndQuarter|第2四半期末の配当予想|
|ForecastDividendPerShare3rdQuarter|第3四半期末の配当予想|
|ForecastDividendPerShareAnnual|通期の配当予想の合計値|
|ForecastDividendPerShareFiscalYearEnd|期末の配当予想|
|ForecastEarningsPerShare|1株当たり当期純利益の通期予想|
|ForecastNetSales|売上高_通期予想|
|ForecastOperatingProfit|営業利益_通期予想|
|ForecastOrdinaryProfit|経常利益_通期予想|
|ForecastProfit|当期純利益_通期予想|
|LocalCode|株式銘柄コード|
|MaterialChangesInSubsidiaries|期中における重要な子会社の異動|
|NetSales|売上高|
|NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock|期末発行済株式数|
|NumberOfTreasuryStockAtTheEndOfFiscalYear|期末自己株式数|
|OperatingProfit|営業利益|
|OrdinaryProfit|経常利益|
|Profit|当期純利益|
|ResultDividendPerShare1stQuarter|第1四半期末の配当実績|
|ResultDividendPerShare2ndQuarter|第2四半期末の配当実績|
|ResultDividendPerShare3rdQuarter|第3四半期末の配当実績|
|ResultDividendPerShareAnnual|通期の配当実績の合計|
|ResultDividendPerShareFiscalYearEnd|期末の配当実績|
|RetrospectiveRestatement|修正再表示|
|TotalAssets|総資産|
|TypeOfCurrentPeriod|当会計期間の種類|
|TypeOfDocument|書類種別|


In [ ]:
df_fins = cli.get_fins_statements(code=8697)

# データフレームを確認
df_fins.info()

print("\n")

# データフレームを描画
df_fins


<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 28
Data columns (total 43 columns):
 #   Column                                                                        Non-Null Count  Dtype         
---  ------                                                                        --------------  -----         
 0   DisclosureNumber                                                              29 non-null     object        
 1   DisclosedDate                                                                 29 non-null     datetime64[ns]
 2   ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements                 29 non-null     object        
 3   AverageNumberOfShares                                                         29 non-null     object        
 4   BookValuePerShare                                                             29 non-null     object        
 5   ChangesBasedOnRevisionsOfAccountingStandard                                   29 non-null     

,DisclosureNumber,DisclosedDate,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,AverageNumberOfShares,BookValuePerShare,ChangesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,CurrentFiscalYearEndDate,CurrentFiscalYearStartDate,CurrentPeriodEndDate,DisclosedTime,DisclosedUnixTime,EarningsPerShare,Equity,EquityToAssetRatio,ForecastDividendPerShare1stQuarter,ForecastDividendPerShare2ndQuarter,ForecastDividendPerShare3rdQuarter,ForecastDividendPerShareAnnual,ForecastDividendPerShareFiscalYearEnd,ForecastEarningsPerShare,ForecastNetSales,ForecastOperatingProfit,ForecastOrdinaryProfit,ForecastProfit,LocalCode,MaterialChangesInSubsidiaries,NetSales,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,OperatingProfit,OrdinaryProfit,Profit,ResultDividendPerShare1stQuarter,ResultDividendPerShare2ndQuarter,ResultDividendPerShare3rdQuarter,ResultDividendPerShareAnnual,ResultDividendPerShareFiscalYearEnd,RetrospectiveRestatement,TotalAssets,TypeOfCurrentPeriod,TypeOfDocument
0,20170127483465,2017-01-30,,547924549,,false,false,false,2017-03-31,2016-04-01,2016-12-31,12:00:00,1485745200.0,58.1,260297000000,0.006,,,,42.0,21.0,70.32,106000000000,54000000000,55500000000,38500000000,86970,false,80663000000,549069100,2754052,44814000000,45783000000,31832000000,－,21.0,－,,,,40450677000000,3Q,3QFinancialStatements_Consolidated_IFRS
1,20170321424149,2017-03-22,,,,,,,2017-03-31,2016-04-01,2017-03-31,14:00:00,1490158800.0,,,,,－,－,47.0,26.0,76.39,107000000000,59000000000,60000000000,41800000000,86970,,,,,,,,,,,,,,,FY,ForecastRevision
2,20170427453686,2017-04-28,,547051034,205.5,false,false,false,2017-03-31,2016-04-01,2017-03-31,12:00:00,1493348400.0,77.0,263770000000,0.006,－,24.0,－,48.0,24.0,78.64,112000000000,60000000000,61500000000,42500000000,86970,false,107885000000,549069100,8636052,59377000000,60604000000,42124000000,－,21.0,－,47.0,26.0,,41288932000000,FY,FYFinancialStatements_Consolidated_IFRS
3,20170727441140,2017-07-28,,537208664,,false,false,false,2018-03-31,2017-04-01,2017-06-30,12:00:00,1501210800.0,20.24,252425000000,0.007,,24.0,－,48.0,24.0,79.27,112000000000,60000000000,61500000000,42500000000,86970,false,27876000000,549069100,13303873,16440000000,16630000000,10874000000,－,,,,,,37878487000000,1Q,1QFinancialStatements_Consolidated_IFRS
4,20171027401390,2017-10-30,,536483009,,false,false,false,2018-03-31,2017-04-01,2017-09-30,12:00:00,1509332400.0,42.3,264947000000,0.006,,,－,48.0,24.0,79.27,110000000000,60000000000,61500000000,42500000000,86970,false,55762000000,549069100,13303807,32808000000,33554000000,22692000000,－,24.0,,,,,42523504000000,2Q,2QFinancialStatements_Consolidated_IFRS
5,20180130459766,2018-01-31,,536242907,,false,false,false,2018-03-31,2017-04-01,2017-12-31,12:00:00,1517367600.0,69.36,267177000000,0.007,,,,48.0,24.0,79.27,110000000000,60000000000,61500000000,42500000000,86970,false,87861000000,549069100,13303754,53090000000,54042000000,37195000000,－,24.0,－,,,,37987261000000,3Q,3QFinancialStatements_Consolidated_IFRS
6,20180322494488,2018-03-22,,,,,,,2018-03-31,2017-04-01,2018-03-31,12:00:00,1521687600.0,,,,,－,－,57.0,33.0,93.26,120000000000,71000000000,72500000000,50000000000,86970,,,,,,,,,,,,,,,FY,ForecastRevision
7,20180426422269,2018-04-27,,536125158,208.99,false,false,false,2018-03-31,2017-04-01,2018-03-31,12:00:00,1524798000.0,94.17,279736000000,0.007,－,27.0,－,54.0,27.0,89.59,123000000000,68000000000,69000000000,48000000000,86970,false,120711000000,536351448,585416,71791000000,72990000000,50484000000,－,24.0,－,67.0,43.0,,41316341000000,FY,FYFinancialStatements_Consolidated_IFRS
8,20180727486943,2018-07-30,,535766032,,true,false,false,2019-03-31,2018-04-01,2018-06-30,12:00:00,1532919600.0,21.69,268065000000,0.007,,27.0,－,54.0,27.0,89.59,123000000000,68000000000,69000000000,48000000000,86970,false,29562000000,536351448,585416,17413000000,17609000000,11623000000,－,,,,,,358421

In [ ]:
# 欠損値の確認
df_fins[df_fins.isnull().any(axis=1)]


,DisclosureNumber,DisclosedDate,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,AverageNumberOfShares,BookValuePerShare,ChangesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,CurrentFiscalYearEndDate,CurrentFiscalYearStartDate,CurrentPeriodEndDate,DisclosedTime,DisclosedUnixTime,EarningsPerShare,Equity,EquityToAssetRatio,ForecastDividendPerShare1stQuarter,ForecastDividendPerShare2ndQuarter,ForecastDividendPerShare3rdQuarter,ForecastDividendPerShareAnnual,ForecastDividendPerShareFiscalYearEnd,ForecastEarningsPerShare,ForecastNetSales,ForecastOperatingProfit,ForecastOrdinaryProfit,ForecastProfit,LocalCode,MaterialChangesInSubsidiaries,NetSales,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,OperatingProfit,OrdinaryProfit,Profit,ResultDividendPerShare1stQuarter,ResultDividendPerShare2ndQuarter,ResultDividendPerShare3rdQuarter,ResultDividendPerShareAnnual,ResultDividendPerShareFiscalYearEnd,RetrospectiveRestatement,TotalAssets,TypeOfCurrentPeriod,TypeOfDocument


取得したデータフレームには空欄の箇所が多くみられるが、Null値は存在しない。  
本APIのデータは、各企業が適時開示したXBRLデータに基づき作成しており、当該XBRLデータにおいて、データの無い箇所には空文字が入っていることから、本APIにおいても空欄となっている箇所が多くある。

### ひとくちメモ集

- 決算短信の"空文字"と"０"と"ー（バー）"について

決算短信の各項目のうち、０とある場合には数値が存在する項目であるが、百万円単位としたときに四捨五入等で切り捨てられた場合が該当します。  
（例）売上高0のとき（売上高0.2百万円）

一方、ー（バー）となる場合は、数値が存在しない項目という場合が該当します。  
（例）売上高ーのとき（売上高0円）

これらについてはよくまとめられた記事が[こちら](https://kabushikisoumu.com/less-than-unit)にありました！
もしよろしければ、ご参考ください。

## 若干のデータ分析

In [ ]:
# 2022年4月1日〜2022年6月30日の財務情報を取得
df_fins = cli.get_statements_range(start_dt="20220401", end_dt="20220630")


In [ ]:
# 書類種別の項目を表示
df_fins["TypeOfDocument"].value_counts()


FYFinancialStatements_Consolidated_JP         2162
ForecastRevision                               795
1QFinancialStatements_Consolidated_JP          487
FYFinancialStatements_NonConsolidated_JP       293
2QFinancialStatements_Consolidated_JP          239
NumericalCorrection                            214
3QFinancialStatements_Consolidated_JP          200
FYFinancialStatements_Consolidated_IFRS        168
1QFinancialStatements_NonConsolidated_JP        99
2QFinancialStatements_NonConsolidated_JP        69
3QFinancialStatements_NonConsolidated_JP        67
1QFinancialStatements_Consolidated_IFRS         56
FYFinancialStatements_REIT                      29
2QFinancialStatements_Consolidated_IFRS         14
FYFinancialStatements_Consolidated_US            9
3QFinancialStatements_Consolidated_IFRS          8
ForecastRevision_REIT                            4
FYFinancialStatements_NonConsolidated_IFRS       2
1QFinancialStatements_Consolidated_US            2
Name: TypeOfDocument, dtype: in

書類種別には、連結（Consolidated）や非連結（NonConsolidated）決算、予想の修正（ForecastRevision）・数値の訂正（NumericalCorrection）の開示等があることがわかる。

In [ ]:
# 会計期間の種類を表示
df_fins["TypeOfCurrentPeriod"].value_counts()


FY    3609
1Q     655
2Q     372
3Q     281
Name: TypeOfCurrentPeriod, dtype: int64

4月から6月までの間に通期の開示を行う会社が多いことがわかる  

In [ ]:
# 事業年度の終了日を表示
df_fins_onlyFY = df_fins[df_fins["TypeOfDocument"].str.contains(
    "FYFinancialStatements")]
df_fins_onlyFY["CurrentPeriodEndDate"].value_counts()


2022-03-31    2359
2022-02-28     222
2022-04-30      50
2022-03-20      14
2022-02-20       5
2022-05-15       3
2022-05-31       3
2022-03-15       2
2021-12-31       2
2022-04-20       2
2022-05-20       1
Name: CurrentPeriodEndDate, dtype: int64

さらに、特に決算期末が3月末の会社が多い


In [ ]:
# 3月末が事業年度最終日の会社でフィルタ
df_fins_onlyFY_onMar = df_fins_onlyFY[df_fins_onlyFY["CurrentPeriodEndDate"] ==
                                      "2022-03-31"]

# 開示日でグループ化し、開示件数を描画
df_disclosed_date = df_fins_onlyFY_onMar.groupby(
    "DisclosedDate").size().to_frame("Count").reset_index()
px.bar(df_disclosed_date, x="DisclosedDate", y="Count", title="日ごとの開示件数")


ある日に開示が集中していることがわかる（決算期末を迎えてから45日以内に開示しなければならないため）

# マーケット情報 API（毎週第4営業日に週次で更新）
（注）2022/9/27時点の情報です。

2017年1月から足元までの投資部門別売買状況（金額）のデータを取得することができます。  
なお、配信データは[こちらのページ](https://www.jpx.co.jp/markets/statistics-equities/investor-type/index.html)で公表している内容と同一です。  

## API出力データ概要


|変数名|概要|
|---|---|
|Section|取引市場名|
|PublishedDate|本データの公表日|
|StartDate|本データの対象週の週初営業日|
|EndDate|本データの対象週の週末営業日|
|ProprietarySales|自己_売金額|
|ProprietaryPurchases|自己_買金額|
|ProprietaryTotal|自己_売買金額合計|
|ProprietaryBalance|自己_売買差引金額|
|BrokerageSales|委託_売金額|
|BrokeragePurchases|委託_買金額|
|BrokerageTotal|委託_売買金額合計|
|BrokerageBalance|委託_売買差引金額|
|TotalSales|総合計_売金額|
|TotalPurchases|総合計_買金額|
|TotalTotal|総合計_売買金額合計
|TotalBalance|総合計_売買差引金額|
|IndividualsSales|国内個人投資家_売金額|
|IndividualsPurchases|国内個人投資家_買金額|
|IndividualsTotal|国内個人投資家_売買金額合計|
|IndividualsBalance|国内個人投資家_売買差引金額|
|ForeignersSales|海外投資家_売金額|
|ForeignersPurchases|海外投資家_買金額|
|ForeignersTotal|海外投資家_売買金額合計|
|ForeignersBalance|海外投資家_売買差引金額|
|SecuritiesCosSales|証券会社_売金額|
|SecuritiesCosPurchases|証券会社_買金額|
|SecuritiesCosTotal|証券会社_売買金額合計|
|SecuritiesCosBalance|証券会社_売買差引金額|
|InvestmentTrustsSales|投資信託_売金額|
|InvestmentTrustsPurchases|投資信託_買金額|
|InvestmentTrustsTotal|投資信託_売買金額合計|
|InvestmentTrustsBalance|投資信託_売買差引金額|
|BusinessCosSales|事業法人_売金額|
|BusinessCosPurchases|事業法人_買金額|
|BusinessCosTotal|事業法人_売買金額合計|
|BusinessCosBalance|事業法人_売買差引金額|
|OtherCosSales|その他法人_売金額|
|OtherCosPurchases|その他法人_買金額|
|OtherCosTotal|その他法人_売買金額合計|
|OtherCosBalance|その他法人_売買差引金額|
|InsuranceCosSales|生損保_売金額|
|InsuranceCosPurchases|生損保_買金額|
|InsuranceCosTotal|生損保_売買金額合計|
|InsuranceCosBalance|生損保_売買差引金額|
|CityBKsRegionalBKsEtcSales|都銀・地銀_売金額|
|CityBKsRegionalBKsEtcPurchases|都銀・地銀_買金額|
|CityBKsRegionalBKsEtcTotal|都銀・地銀_売買金額合計|
|CityBKsRegionalBKsEtcBalance|都銀・地銀_売買差引金額|
|TrustBanksSales|信託銀行_売金額|
|TrustBanksPurchases|信託銀行_買金額|
|TrustBanksTotal|信託銀行_売買金額合計|
|TrustBanksBalance|信託銀行_売買差引金額
|OtherFinancialInstitutionsSales|その他金融機関_売金額|
|OtherFinancialInstitutionsPurchases|その他金融機関_買金額|
|OtherFinancialInstitutionsTotal|その他金融機関_売買金額合計|
|OtherFinancialInstitutionsBalance|その他金融機関_売買差引金額|

In [ ]:
df_trades_spec = cli.get_markets_trades_spec()

# データフレームを確認
df_trades_spec.info()

print("\n")

# データフレームを描画
df_trades_spec


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1461 entries, 3 to 1459
Data columns (total 56 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Section                              1461 non-null   object        
 1   PublishedDate                        1461 non-null   datetime64[ns]
 2   StartDate                            1461 non-null   datetime64[ns]
 3   EndDate                              1461 non-null   datetime64[ns]
 4   ProprietarySales                     1461 non-null   float64       
 5   ProprietaryPurchases                 1461 non-null   float64       
 6   ProprietaryTotal                     1461 non-null   float64       
 7   ProprietaryBalance                   1461 non-null   float64       
 8   BrokerageSales                       1461 non-null   float64       
 9   BrokeragePurchases                   1461 non-null   float64       
 10  BrokerageTot

,Section,PublishedDate,StartDate,EndDate,ProprietarySales,ProprietaryPurchases,ProprietaryTotal,ProprietaryBalance,BrokerageSales,BrokeragePurchases,BrokerageTotal,BrokerageBalance,TotalSales,TotalPurchases,TotalTotal,TotalBalance,IndividualsSales,IndividualsPurchases,IndividualsTotal,IndividualsBalance,ForeignersSales,ForeignersPurchases,ForeignersTotal,ForeignersBalance,SecuritiesCosSales,SecuritiesCosPurchases,SecuritiesCosTotal,SecuritiesCosBalance,InvestmentTrustsSales,InvestmentTrustsPurchases,InvestmentTrustsTotal,InvestmentTrustsBalance,BusinessCosSales,BusinessCosPurchases,BusinessCosTotal,BusinessCosBalance,OtherCosSales,OtherCosPurchases,OtherCosTotal,OtherCosBalance,InsuranceCosSales,InsuranceCosPurchases,InsuranceCosTotal,InsuranceCosBalance,CityBKsRegionalBKsEtcSales,CityBKsRegionalBKsEtcPurchases,CityBKsRegionalBKsEtcTotal,CityBKsRegionalBKsEtcBalance,TrustBanksSales,TrustBanksPurchases,TrustBanksTotal,TrustBanksBalance,OtherFinancialInstitutionsSales,OtherFinancialInstitutionsPurchases,OtherFinancialInstitutionsTotal,OtherFinancialInstitutionsBalance
3,TSE1st,2017-01-13,2017-01-04,2017-01-06,1.311271e+09,1.453327e+09,2.764598e+09,142055504.0,7.165529e+09,7.030020e+09,1.419555e+10,-135509151.0,8.476800e+09,8.483346e+09,1.696015e+10,6546353.0,1.401712e+09,1.161801e+09,2.563513e+09,-239910460.0,5.094892e+09,5.317152e+09,1.041204e+10,222260039.0,76381455.0,61700100.0,138081555.0,-14681355.0,168705109.0,124389642.0,293094751.0,-44315467.0,71217959.0,63526641.0,134744600.0,-7691318.0,10745152.0,15687836.0,26432988.0,4942684.0,15926202.0,9831555.0,25757757.0,-6094647.0,10606789.0,8843871.0,19450660.0,-1762918.0,292932297.0,245322795.0,538255092.0,-47609502.0,22410692.0,21764485.0,44175177.0,-646207.0
2,TSE2nd,2017-01-13,2017-01-04,2017-01-06,1.309361e+07,6.535935e+06,1.962954e+07,-6557672.0,2.092613e+08,2.152348e+08,4.244960e+08,5973519.0,2.223549e+08,2.217707e+08,4.441256e+08,-584153.0,1.224825e+08,1.239568e+08,2.464393e+08,1474255.0,7.623919e+07,8.047202e+07,1.567112e+08,4232828.0,6051426.0,5778242.0,11829668.0,-273184.0,600795.0,694595.0,1295390.0,93800.0,2139840.0,1980229.0,4120069.0,-159611.0,314555.0,494071.0,808626.0,179516.0,19636.0,12986.0,32622.0,-6650.0,36866.0,2301.0,39167.0,-34565.0,1328241.0,1807118.0,3135359.0,478877.0,48202.0,36455.0,84657.0,-11747.0
1,TSEJASDAQ,2017-01-13,2017-01-04,2017-01-06,4.949104e+06,4.193588e+06,9.142692e+06,-755516.0,1.416962e+08,1.422985e+08,2.839947e+08,602320.0,1.466453e+08,1.464921e+08,2.931374e+08,-153196.0,1.000472e+08,9.806762e+07,1.981148e+08,-1979577.0,3.205425e+07,3.438530e+07,6.643955e+07,2331056.0,4653575.0,4334909.0,8988484.0,-318666.0,1099405.0,1336156.0,2435561.0,236751.0,1437706.0,1453605.0,2891311.0,15899.0,244023.0,342691.0,586714.0,98668.0,14484.0,318.0,14802.0,-14166.0,51072.0,2026.0,53098.0,-49046.0,2077054.0,2271613.0,4348667.0,194559.0,17444.0,104286.0,121730.0,86842.0
4,TSEMothers,2017-01-13,2017-01-04,2017-01-06,1.110837e+07,8.629083e+06,1.973745e+07,-2479283.0,3.014538e+08,3.042051e+08,6.056589e+08,2751341.0,3.125621e+08,3.128342e+08,6.253963e+08,272058.0,2.042972e+08,2.023668e+08,4.066639e+08,-1930418.0,8.185699e+07,8.562572e+07,1.674827e+08,3768728.0,7397443.0,6906340.0,14303783.0,-491103.0,805816.0,1837203.0,2643019.0,1031387.0,3524194.0,3405887.0,6930081.0,-118307.0,870179.0,489709.0,1359888.0,-380470.0,6691.0,4024.0,10715.0,-2667.0,10205.0,36575.0,46780.0,26370.0,2681157.0,3526373.0,6207530.0,845216.0,3918.0,6523.0,10441.0,2605.0
0,TokyoNagoya,2017-01-13,2017-01-04,2017-01-06,1.340438e+09,1.472728e+09,2.813166e+09,132290383.0,7.819145e+09,7.692945e+09,1.551209e+10,-126199641.0,9.159583e+09,9.165674e+09,1.832526e+10,6090742.0,1.829215e+09,1.586814e+09,3.416028e+09,-242401144.0,5.285399e+09,5.518022e+09,1.080342e+10,232623007.0,94500941.0,78768114.0,173269055.0,-15732827.0,171266407.0,128280714.0,299547121.0,-42985693.0,78352044.0,70374877.0,148726921.0,-7977167.0,12185795.0,17106799.0,29292594.0,4921004.0,15968397.0,9848883.0,25817280.0,-6119514.0

In [ ]:
# 欠損値の確認
df_trades_spec[df_trades_spec.isnull().any(axis=1)]


,Section,PublishedDate,StartDate,EndDate,ProprietarySales,ProprietaryPurchases,ProprietaryTotal,ProprietaryBalance,BrokerageSales,BrokeragePurchases,BrokerageTotal,BrokerageBalance,TotalSales,TotalPurchases,TotalTotal,TotalBalance,IndividualsSales,IndividualsPurchases,IndividualsTotal,IndividualsBalance,ForeignersSales,ForeignersPurchases,ForeignersTotal,ForeignersBalance,SecuritiesCosSales,SecuritiesCosPurchases,SecuritiesCosTotal,SecuritiesCosBalance,InvestmentTrustsSales,InvestmentTrustsPurchases,InvestmentTrustsTotal,InvestmentTrustsBalance,BusinessCosSales,BusinessCosPurchases,BusinessCosTotal,BusinessCosBalance,OtherCosSales,OtherCosPurchases,OtherCosTotal,OtherCosBalance,InsuranceCosSales,InsuranceCosPurchases,InsuranceCosTotal,InsuranceCosBalance,CityBKsRegionalBKsEtcSales,CityBKsRegionalBKsEtcPurchases,CityBKsRegionalBKsEtcTotal,CityBKsRegionalBKsEtcBalance,TrustBanksSales,TrustBanksPurchases,TrustBanksTotal,TrustBanksBalance,OtherFinancialInstitutionsSales,OtherFinancialInstitutionsPurchases,OtherFinancialInstitutionsTotal,OtherFinancialInstitutionsBalance


## 若干のデータ分析

In [5]:
# 2022年4月以降のプライム市場の投資部門別データを取得
df_trades_spec = cli.get_markets_trades_spec(section=jquantsapi.MARKET_API_SECTIONS.TSEPrime,
                                             from_yyyymmdd=20220408)

# 各属性のうち、Balance（売り買いの差し引き金額）の項目のみ取得
columns_balance = df_trades_spec.columns[df_trades_spec.columns.str.contains(
    "Balance")]
df_trades_spec_only_balance = df_trades_spec[columns_balance].copy()

# 投資部門別データの公表日を選択した項目のデータフレームへ転記
df_trades_spec_only_balance.loc[:, "StartDate"] = df_trades_spec[
    "StartDate"].copy()
df_trades_spec_only_balance.set_index("StartDate", inplace=True)

# 海外投資家、個人、投資信託、信託銀行の4つの属性のみ描画
fig = px.bar(df_trades_spec_only_balance,
             x=df_trades_spec_only_balance.index,
             y=[
                 "ForeignersBalance", "IndividualsBalance",
                 "InvestmentTrustsBalance", "TrustBanksBalance"
             ],
             barmode="group")
fig.update_xaxes(type="category")
fig.show()


In [11]:
# 日ごとだと見にくいため、月単位で合計してみる
df_monthly = df_trades_spec_only_balance.resample("M").sum()
fig = px.bar(df_monthly,
       x=df_monthly.index,
       y=[
           "ForeignersBalance", "IndividualsBalance",
           "InvestmentTrustsBalance", "TrustBanksBalance"
       ],
       barmode="group")
fig.update_xaxes(
    dtick="M1",
    tickformat="%b %Y",
    tick0="2022-04-30")
fig.show()


2022年4月に海外投資家が大きく買い越しているが、5月以降はほぼ売り越しとなっていそう。
- 2022年4月以前の状況はどうか？
- 今回はプライム市場でフィルタしているが、他の市場区分だとどうか？

# FinVizのTreeMapみたいなやつ

- 各APIを統合して利用した分析として、FinVizのTreeMapのようなものを作ってみる

## 時価総額を算出するため、各銘柄の期末発行済み株式数を取得

In [13]:
# 2022年初来の財務情報取得
df_fins = cli.get_statements_range(start_dt="20220101")


# 財務情報データから取得したい項目（株式銘柄コード、期末発行済株式数）を定義
fins_columns = [
    "LocalCode",
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"
]

# 財務情報テータから必要なデータのみ取得
df_treasury_stock = df_fins[df_fins[
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"]
                            != ""][fins_columns]

# 列名の変更（LocalCode -> Code）
df_treasury_stock = df_treasury_stock.rename(columns={"LocalCode": "Code"})


# 期末発行済株式数を文字列から整数に変換
df_treasury_stock[
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"] = df_treasury_stock[
        "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"].astype("int")

# 期末発行済株式数を銘柄毎にグループ化して平均
df_treasury_stock = df_treasury_stock.groupby(
    by="Code").mean().reset_index()


## 33業種のそれぞれについて、時価総額Top5の企業を取得

In [14]:
# 昨日の全銘柄株価を取得
df_prices = cli.get_prices_daily_quotes(date_yyyymmdd=20220927)

# 銘柄一覧を取得
df_info = cli.get_list()

# 銘柄一覧と全銘柄株価を株式銘柄コードで結合
df_merge = pd.merge(df_prices, df_info, how="inner", on="Code")

# さらに先程取得した期末発行済み株式数と結合
df_merge = pd.merge(df_merge, df_treasury_stock, how="inner", on="Code")

# 時価総額をデータフレームに追加
df_merge["Capitalization"] = df_merge["AdjustmentClose"] * df_merge[
    "NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock"]

# 時価総額及び業種コードでソート
df_merge = df_merge.sort_values(by=["Capitalization"], ascending=False)

# 業種名毎の上位５社を抽出
df_top5 = df_merge.groupby("SectorName").head(5)

# 業種その他（ETF等）を除外
df_top5 = df_top5[df_top5["SectorCode"] != "9999"]

# 各業種の時価総額Top5の株式銘柄コードを取得
top5_codes = df_top5["Code"]


## 33業種のそれぞれについて、時価総額Top5の企業の約1か月（20営業日）のリターンを取得

In [15]:
# 7月末から8月末までの約１か月分の全銘柄の株価情報を取得
df_1month_prices = cli.get_price_range(start_dt="20220729", end_dt="20220831") 

# 各業種の時価総額Top5のみの調整済み終値を取得
df_top5_1month_prices = df_1month_prices[df_1month_prices["Code"].isin(
    top5_codes)][["Code", "Date", "AdjustmentClose"]]

# 約1か月のリターン計算する関数
def calc_return(df, code):
    df = df[df["Code"] == code]
    df["Return"] = df["AdjustmentClose"].pct_change(20)
    return df[["Code", "Return"]].tail(1).to_dict(orient="records")

# 各銘柄の約1か月のリターンを作成
returns_list = []

for code in top5_codes:
    df = calc_return(df_top5_1month_prices, code)
    returns_list.extend(df)

# リターンのデータフレームを作成
df_returns = pd.DataFrame(returns_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## 業種、企業名、時価総額、リターンからtreemapを描画

In [16]:
# リターンのデータフレームを結合
df_top5_with_returns = pd.merge(df_top5, df_returns, how="inner", on="Code")

# treemapを描画
fig = px.treemap(df_top5_with_returns,
                 path=[px.Constant("TSE"), "SectorName", "CompanyName"],
                 values="Capitalization",
                 color="Return",
                 color_continuous_scale=["red", "white", "green"],
                 color_continuous_midpoint=0)
fig.update_layout(margin=dict(t=50, l=25, r=25, b=25))
fig.show()


# 今後のアップデート予定
- 銘柄情報一覧に17業種及び規模コードの追加
- 過去時点の銘柄情報一覧が取得できるように
- TOPIX指数の追加

その他、こんなデータが欲しいというご要望や、こんなデータありますか？といった質問もどしどし受け付けております！！

discordの`#j-quants-api`チャンネルに送ってください！！  
よろしくお願いいたします。